In [1]:
# torch dependencies
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": device} # set tensors to GPU, if multiple GPUs please set cuda:x properly


torch.set_printoptions(precision=3)

# NN dependencies
from torch.utils.data import DataLoader

from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP, FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.optim.optimize import optimize_acqf,optimize_acqf_mixed,optimize_acqf_discrete_local_search
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.hypervolume import infer_reference_point
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

In [9]:
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [3]:
from tensorflow.keras.models import load_model

In [4]:
import os
os.chdir('./sources')
from Cal_SA import SA_score
from Cal_TC import pre_TC
from utility import Initial_X,cover_torch,cover_numpy,X_info

In [5]:
def optimize_qnehvi(Problem,N_BATCH, BATCH_SIZE, verbose=False):
    print("Optimizing with qNEHVI")
    t0 = time.time()
    # some initializing 
    torch.manual_seed(Problem.seed) # gives a consistent seed based on the trial number
    hv=Hypervolume(ref_point=-Problem.ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    # establishing constraints
    Choise_list=[]
    for i in range (Problem.n_var):
        Choise_list.append(torch.tensor([0.,1.]))
    # generate initial training data for that run
    TXtrain=torch.tensor(Problem.Initial_Xdata,dtype=torch.float64)
    TYtrain=Problem.evaluate(TXtrain)
    # computing HV of initial candidate list
    pareto_mask = is_non_dominated(TYtrain) # check for 2nd criteria: non-dominated, meaning new pareto optimal
    pareto_y = TYtrain[pareto_mask] # take only points that fit the 2nd check
    volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
    hvs.append(volume)
    # define and train surrogate models for objective and constraint
    models = []
    for i in range(TYtrain.shape[-1]):
        models.append(SingleTaskGP(TXtrain, TYtrain[..., i : i + 1]))
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    
    ##########    
    # original location for an extra HV check wrt to initial samples
    ########## 
    # training loop for N_BATCH iterations
    for iteration in range(1, N_BATCH + 1): 
        t3 = time.time()
        # fit the surrogate model
        fit_gpytorch_model(mll)    
        # define the acqusition function 
        acq_func = qNoisyExpectedHypervolumeImprovement(model=model,
            ref_point=-Problem.ref_point, # for computing HV, must flip for BoTorch
            X_baseline=TXtrain, # feed total list of train_x for this current iteration
            sampler=SobolQMCNormalSampler(sample_shape=128),  # determines how candidates are randomly proposed before selection
            objective=IdentityMCMultiOutputObjective(outcomes=np.arange(Problem.n_obj).tolist()), # optimize first n_obj col 
            prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        # propose candidates given defined qNEHVI acq func given model 
        new_x,_= optimize_acqf_discrete_local_search(
                            acq_function=acq_func,
                            discrete_choices=Choise_list, # since train_x was normalized                    
                            q=BATCH_SIZE, # no of candidates to propose in parallel
                            num_restarts=2, # no of restarts if q candidates fail to show improvement
                            raw_samples=256,  # pool of samples to choose the starting points from
                            options={"batch_limit": 5, "maxiter": 200}, # default arguments
                            )
        new_obj=Problem.evaluate(new_x)
        TXtrain=torch.cat([TXtrain, new_x])
        TYtrain=torch.cat([TYtrain, cover_torch(new_obj)])
        ##########
        # computing HV of current candidate list
        pareto_mask = is_non_dominated(TYtrain) # check for 2nd criteria: non-dominated, meaning new pareto optimal
        pareto_y = TYtrain[pareto_mask] # take only points that fit the 2nd check
        volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        hvs.append(volume)
        # update the surrogate models for next iteration
        models = []
        for i in range(TYtrain.shape[-1]):
            models.append(SingleTaskGP(TXtrain, TYtrain[..., i : i + 1]))
        model = ModelListGP(*models)
        mll = SumMarginalLogLikelihood(model.likelihood, model)
        
        t4 = time.time()
        if verbose:
            print(
                    f"Batch {iteration:>2} of {N_BATCH}: Hypervolume = "
                    f"{hvs[-1]:>4.2f}, "
                    f"time = {t4-t3:>4.2f}s.\n"
                    , end="")
        del new_x, new_obj
        torch.cuda.empty_cache()
        t1 = time.time()
        print(f"Time taken in total: {t1-t0:>4.2f}s.") 
    # returns the HV score across iterations, total training set as an array
    return hvs, torch.hstack([TXtrain, TYtrain]).cpu().numpy() 

In [6]:
TC_model=load_model(r"TC_DNN.h5",compile=False)
df=pd.read_csv(r'smi.csv')
chromosome=df["chromosome"]
selected_cor=pd.read_csv(r'select_keys.csv',index_col=0)
selected_keys=np.array(selected_cor["key"])
selected_Corr_df=np.array(selected_cor["Index"])

In [13]:
class Problem(torch.nn.Module):
    # must define these!
    n_var = 25
    n_obj = 2 
    n_random=10
    seed=0
    TC_model=TC_model
    selected_keys=selected_keys
    chromosome=chromosome
    Initial_Xdata=Initial_X(n_var,n_random,seed)
    ref_point = torch.tensor([0.0,10.0]) 
    def evaluate(X,n_var0=n_var):
        X=cover_numpy(X).reshape(-1,n_var0)
        res=[]
        for x in X:
            tc=pre_TC(x,model=TC_model,selected_keys=selected_keys,chromosome=chromosome)
            TC_=tc[0]
            SA_=SA_score(tc[2])*(-1) 
            res.append(TC_)
            res.append(SA_)
        #print (res)
        return cover_torch (np.array(res).reshape(-1,2)) #Uniformity as a maximum problem

In [14]:
hvs_qnehvi, train_qnehvi = optimize_qnehvi(Problem,N_BATCH=50, BATCH_SIZE=10, verbose=True)

Optimizing with qNEHVI
Batch  1 of 50: Hypervolume = 4.22, time = 5.67s.
Time taken in total: 9.64s.
Batch  2 of 50: Hypervolume = 4.36, time = 5.99s.
Time taken in total: 15.63s.
Batch  3 of 50: Hypervolume = 4.67, time = 6.05s.
Time taken in total: 21.69s.
Batch  4 of 50: Hypervolume = 4.87, time = 6.65s.
Time taken in total: 28.34s.
Batch  5 of 50: Hypervolume = 4.96, time = 6.88s.
Time taken in total: 35.22s.
Batch  6 of 50: Hypervolume = 4.96, time = 6.16s.
Time taken in total: 41.37s.
Batch  7 of 50: Hypervolume = 5.00, time = 6.74s.
Time taken in total: 48.11s.
Batch  8 of 50: Hypervolume = 5.00, time = 7.55s.
Time taken in total: 55.66s.
Batch  9 of 50: Hypervolume = 5.02, time = 7.30s.
Time taken in total: 62.96s.
Batch 10 of 50: Hypervolume = 5.02, time = 7.68s.
Time taken in total: 70.64s.
Batch 11 of 50: Hypervolume = 5.02, time = 7.52s.
Time taken in total: 78.16s.
Batch 12 of 50: Hypervolume = 5.04, time = 7.58s.
Time taken in total: 85.74s.
Batch 13 of 50: Hypervolume = 

In [15]:
gen=[]
for i in range(51):
    for j in range (10):
        gen.append(i)
df_gen=pd.DataFrame(gen,columns=["Gen"])   

In [16]:
res=[X_info(arr) for arr in train_qnehvi[:,:-2]]
df_res=pd.concat([pd.DataFrame(np.array(res, dtype=object).reshape(-1,2)),pd.DataFrame(train_qnehvi)],axis=1)
df_res.columns=["Serial","SMILES"]+["Bit_"+str(x) for x in range(25)]+["TC","SA"]
df_res=df_res.join(df_gen)
df_dup=df_res.drop_duplicates(subset="SMILES")
df_HV=pd.DataFrame(np.array(hvs_qnehvi),columns=["HV"])

In [17]:
#Polymes designed by MOBO
df_dup

,Serial,SMILES,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,...,Bit_18,Bit_19,Bit_20,Bit_21,Bit_22,Bit_23,Bit_24,TC,SA,Gen
0,"[5, 18, 7, 1, 4]",O=C(NCCOCCCCCCc1ccc([*])[nH]1)Nc1ccc2cc([*])cc...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.260,-3.336085,0
1,"[22, 1, 4, 8, 28]",O=C(OC(=O)c1ccc(C(=O)N[*])cc1)c1ccc(CCCCCCc2nc...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.277,-3.543302,0
2,"[24, 9, 13, 21, 11]",O=C(C=CC(=O)c1ccc(Nc2ccc(Nc3nc4cc(C(=O)c5ccc(C...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.490,-3.505829,0
3,"[6, 9, 29, 19, 6]",O=C(NNC(=O)c1ccc2c(c1)Cc1cc([*])ccc1-2)C(=O)Nc...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.415,-3.585255,0
4,"[16, 18, 28, 28, 3]",O=C(NCNC(=O)c1ccc(C(=O)C(=O)c2ccc(C(=O)Nc3ccc(...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.546,-3.551942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,"[25, 25, 28, 13, 20]",O=C(NN[*])C(=O)Nc1ccc(NC(=O)c2ccc(C(=O)NNC(=O)...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.715,-2.764242,50
506,"[0, 0, 23, 29, 11]",O=C(C=CC(=O)NC(=O)Nc1ccc(NC(=O)C(=O)c2ccc(C(=O...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.553,-3.488854,50
507,"[0, 9, 10, 30, 19]",O=C([*])NNC(=O)n1c(=O)c2cc3c(=O)n(C(=O)C=CC(=O...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.365,-4.101432,50
508,"[0, 7, 19, 28, 21]",O=C(NNC(=O)C(=O)c1ccc(C(=O)Nc2nc3cc([*])ccc3o2...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.388,-3.610613,50


In [18]:
#HVs improvement
df_HV

,HV
0,3.639349
1,4.222101
2,4.361312
3,4.667527
4,4.870332
5,4.959408
6,4.959408
7,5.004693
8,5.004693
9,5.018311


In [19]:
df_HV.to_csv("./results/MOBO_HV.csv")
df_dup.to_csv("./results/MOBO_candidates.csv")